In [5]:
import os
from datetime import datetime

from core.runner import Runner
from core.environment import SingleVertexSpinFoam, StarModelSpinFoam

def train_gfn(
    spin_j, 
    sf_model,
    main_layer_hidden_nodes, 
    branch1_hidden_nodes, 
    branch2_hidden_nodes,
    activation,
    exploration_rate,
    training_fraction_from_back_traj,
    learning_rate,
    batch_size,
    n_iterations,
    evaluation_batch_size,
    generate_samples_every_m_training_samples,
):
    if sf_model == "single_vertex_model":
        spinfoam_model = SingleVertexSpinFoam(spin_j=spin_j)
    elif sf_model == "star_model":
        spinfoam_model = StarModelSpinFoam(spin_j=spin_j)
    else:
        raise ValueError(
            "Spinfoam model not yet implemented. "
            "Custom Spinfoam class can be made."
        )
        
    #training_run_datetime = datetime.now().strftime("%B %d, %Y at %H:%M:%S")
    
    directory_for_generated_samples = (
        f"./data/GFlowNet/gener_samples_during_train/{sf_model}/j_{spin_j}/n_iterations_{n_iterations}/main_layer_hid_nodes_{main_layer_hidden_nodes}/exploration_rate_{exploration_rate}/learning_rate_{learning_rate}/batch_size_{batch_size}"
    )
    
    runner = Runner(
        spinfoam_model=spinfoam_model,
        main_layer_hidden_nodes=main_layer_hidden_nodes,
        branch1_hidden_nodes=branch1_hidden_nodes,
        branch2_hidden_nodes=branch2_hidden_nodes,
        activation=activation,
        exploration_rate=exploration_rate,
        training_fraction_from_back_traj=training_fraction_from_back_traj,
        learning_rate=learning_rate
    )
    
    ave_losses = runner.train_agent(
        batch_size, n_iterations, 
        evaluation_batch_size, generate_samples_every_m_training_samples,
        directory_for_generated_samples
    )
    
    return ave_losses

## Vertex Amplitude

In [11]:
spin_j=3.5 
sf_model="single_vertex_model"
main_layer_hidden_nodes=(30, 20) 
branch1_hidden_nodes=()
branch2_hidden_nodes=()
activation="swish"
exploration_rate=0.5
training_fraction_from_back_traj=0.0
learning_rate=0.0005
batch_size=int(1e2)
n_iterations=int(1e3)
evaluation_batch_size=int(1e5)
generate_samples_every_m_training_samples=int(1e5)


In [12]:
ave_losses = train_gfn(
    spin_j, 
    sf_model,
    main_layer_hidden_nodes, 
    branch1_hidden_nodes, 
    branch2_hidden_nodes,
    activation,
    exploration_rate,
    training_fraction_from_back_traj,
    learning_rate,
    batch_size,
    n_iterations,
    evaluation_batch_size,
    generate_samples_every_m_training_samples,
)

Nth iteration: 1000 Trained on K samples: 100000 Average Loss: 7.5104173708239133


# TODO list

- Write a script on Compute Canada clusters (possibly with GPU offloading) to test the algorithm (at least with the 4-simplex or vertex amplitude model) using different parameters, such as: 
  
  NODES AND LAYERS (neural network architectures):
  (160, 160, 80, 80, 40, 40),
  (160, 160, 40, 40),
  (80, 80, 40, 40),
  (80, 40).
  
  EXPLORATION RATES:
  0, 0.25, 0.5.
  
  LEARNING RATES:
  0.01, 0.001, 0.0001.
